In [ ]:
import mg5qs_import as qs
from pathlib import Path
import os
import numpy as np 
import matplotlib.pyplot as plt

### Vary parameters using API and visual editing tools

This example will demonstrate how MadGraph's built-in capabilities to edit model and run parameters are carried over into mg5qs.

This example includes:
- use of visual card editor
- use of API for editing model parameters
- looping over model parmeters
- associating parameter values with corresponding run 

### 1. Generate a MadGraph framework and use visual card editor

**1.1 Create MadGraph framework and edit run_card**

The visual card editor uses IPyWidgets (native to Jupyter) to edit cards, it is essentially a replacement for vi/gedit/nano/etc. command line file editors. 

In [ ]:
# edit proc_card before using it to generate framework 
qs.edit_card_spec(qs.INPUT_PATH / 'proc_card.dat')
# try changing the output name, currently 'EXAMPLE'

In [ ]:
# generate framework 
output_name, FRAMEWORK_PATH = qs.run_MG5(qs.MG5_PATH, qs.INPUT_PATH, proc_card_name='proc_card.dat')

**1.2 change run parmeters by editing run_card generated allong with the framework**

Each new framework is created with a number of cards (see FRAMEWORK_PATH/Cards). The run_card contains: number of events, random seed, and many more runtime parameters. 

In [ ]:
qs.edit_card(FRAMEWORK_PATH) # by default, edit_card will chose the run_card

**1.3 load param_card using API and change a value**

The param_card can be edited using the visual editor, but it is often preferred to use the API since values can be changed without manual input. Also, some parmeter cards have hundreds or even thousands of lines, so the visual editor is cumbersome. Using the API allows LHE generation to be performed in a loop which changes one or more parameters each time.

In [ ]:
# load param card and look at contents 
card = qs.ParamCard(FRAMEWORK_PATH)
card

In [ ]:
# the ParamCard object stores a dictionary of pandas dataframes
card.dfs().keys() # view keys

In [ ]:
# to find a specifc value, look at a particular dataframe
df_mass = card.dfs()['MASS']
df_mass # notice the key ascocated with Higgs mass is 25

For longer param_cards, search by parameter name this way:

In [ ]:
# this is the standard way of searching a pandas dataframe
df_mass[df_mass['comment'].str.startswith('MH')]

In [ ]:
# call set_value using the block name and key
new_MH = 42
card.set_value('MASS', 25, new_MH)
# reload dataframe and check change 
df_mass = card.dfs()['MASS']
df_mass[df_mass['comment'].str.startswith('MH')]

### 2. Generate LHEs with altered parameters

**2.1 write a loop that changes a parameter value and generates LHEs**

In [ ]:
# loop over values for the Higgs mass and generate LHEs 
for MH in np.linspace(75, 175, 5):
    card.set_value('MASS', 25, MH)  # set the new value 
    print('Working on MH =', MH, '...')
    qs.generate_LHE(card, FRAMEWORK_PATH) # generate LHE with current Higgs mass
print('Done')

### 3. Generate transverse momenta

**3.1 get LHEs**

In [ ]:
LHEs = qs.get_LHEs(FRAMEWORK_PATH)

**3.2 call pT_particle using multithreading**

In [ ]:
import concurrent.futures
cpu_cores = 10  # set an appropriate value based on CPU hardware
results = {}

def process_LHE(LHE, PID=15):
    result =  qs.generate_pT(PID, LHE)
    return LHE.parent.name, result

with concurrent.futures.ProcessPoolExecutor(max_workers=cpu_cores) as executor:
    futures = {executor.submit(process_LHE, LHE): LHE for LHE in LHEs}
    for future in concurrent.futures.as_completed(futures):
        name, result = future.result()
        results[name] = result  

In [ ]:
fig, ax = plt.subplots(len(results), figsize=(7, len(results)*6))

ks = np.sort([key for key in results.keys()]) # construct sorted list of keys 
MH = np.linspace(75, 175, 5)

for i, k in enumerate(ks):
    ax[i].hist(results[k][1], bins=30)
    ax[i].set_title(k+'; Higgs Mass = '+str(MH[i])+'GeV')
    ax[i].set_yscale('log')
    ax[i].set_xlabel("pT (GeV)")
plt.show()